In [1]:
import os
os.environ['OMP_NUM_THREADS']=str(1)

# Import statements

In [2]:
from pytao import Tao
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import re
import io
from os import path,environ
import pandas as pd
import random
from IPython.display import display, clear_output, update_display
import bayes_opt

from pmd_beamphysics import ParticleGroup
#from pmd_beamphysics.statistics import resample_particles
import pmd_beamphysics.statistics

from UTILITY_plotMod import plotMod, slicePlotMod
from UTILITY_linacPhaseAndAmplitude import getLinacMatchStrings, setLinacPhase, setLinacGradientAuto
from UTILITY_modifyAndSaveInputBeam import modifyAndSaveInputBeam
from UTILITY_setLattice import setLattice, getBendkG, getQuadkG, getSextkG

import os
filePath = os.getcwd()
%env FACET2_LATTICE {filePath}
#%env FACET2_LATTICE /Users/nmajik/Documents/SLAC/slacsyncgit/bmadExample

print('Make sure enviornment was set!:  ', environ['FACET2_LATTICE']) 

env: FACET2_LATTICE=/Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Make sure enviornment was set!:   /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao


# Start tao

In [3]:
tao=Tao('-init {:s}/bmad/models/f2_elec/tao.init -noplot'.format(environ['FACET2_LATTICE'])) 
#tao=Tao('-init {:s}/bmad/models/f2_elec/tao.init'.format(environ['FACET2_LATTICE'])) 

#Need to explicitly add non-MARKER places to save beam...
tao.cmd("set beam add_saved_at = DTOTR, XTCAVF, M2EX")

[]

# Beam construction

## Import beam

In [4]:
lastTrackedElement = "end"
tao.cmd(f'set beam_init track_end = {lastTrackedElement}')
#Can see present track_start and track_end values with `show beam`


#Optional: Enable CSR
tao.cmd(f'call {filePath}/bmad/models/f2_elec/scripts/Activate_CSR.tao')
#tao.cmd('csron')
tao.cmd('csroff')

[]

In [5]:
import multiprocessing
print("Number of CPU cores on node:", multiprocessing.cpu_count())

import os

# Check SLURM environment variable
slurm_cpus = os.getenv('SLURM_CPUS_ON_NODE')

# Check PBS environment variable
pbs_cpus = os.getenv('PBS_NP')

# Check SGE environment variable
sge_cpus = os.getenv('NSLOTS')

# Determine the number of allocated cores
allocated_cores = None
if slurm_cpus:
    allocated_cores = int(slurm_cpus)
elif pbs_cpus:
    allocated_cores = int(pbs_cpus)
elif sge_cpus:
    allocated_cores = int(sge_cpus)
else:
    allocated_cores = multiprocessing.cpu_count()  # Fallback to total cores if no job scheduler variables found

# Print the number of allocated cores
print("Number of allocated CPU cores:", allocated_cores)

Number of CPU cores on node: 10
Number of allocated CPU cores: 10


In [6]:
#This loop doesn't work, it keeps hitting all the cores. Instead, I'm going to put this command at the top and rerun manually
# import os

# for coreCount in np.arange(10,0,-1):
#     os.environ['OMP_NUM_THREADS']=str(coreCount)
    
#     inputBeamFilePath = f'{filePath}/beams/nmmToL0AFEND_2bunch_2024-02-16Clean/2024-02-16_2bunch_1e5Downsample_nudgeWeights.h5'
    
    
#     modifyAndSaveInputBeam(
#         inputBeamFilePath,
#         #numMacroParticles = 10000
#     )
    
#     tao.cmd(f'set beam_init position_file={filePath}/beams/activeBeamFile.h5')
#     tao.cmd('reinit beam')
    
#     #Set lattice to my latest default config
#     setLattice(tao)
    
#     import time
#     start_time = time.time()
    
    
#     tao.cmd('set global track_type = beam') #set "track_type = single" to return to single particle
#     tao.cmd('set global track_type = single') #return to single to prevent accidental long re-evaluation
    
#     print(f"{os.environ['OMP_NUM_THREADS']}, {time.time() - start_time}")

In [7]:
inputBeamFilePath = f'{filePath}/beams/nmmToL0AFEND_2bunch_2024-02-16Clean/2024-02-16_2bunch_1e5Downsample_nudgeWeights.h5'


modifyAndSaveInputBeam(
    inputBeamFilePath,
    #numMacroParticles = 10000
)

tao.cmd(f'set beam_init position_file={filePath}/beams/activeBeamFile.h5')
tao.cmd('reinit beam')

#Set lattice to my latest default config
setLattice(tao)

import time
start_time = time.time()


tao.cmd('set global track_type = beam') #set "track_type = single" to return to single particle
tao.cmd('set global track_type = single') #return to single to prevent accidental long re-evaluation

print(f"{os.environ['OMP_NUM_THREADS']}, {time.time() - start_time}")

1, 366.8837628364563
